# Advanced FRED Data Analysis

This notebook demonstrates advanced analysis techniques using FRED economic data.

**Compatible with Google Colaboratory**

## Setup

In [ ]:
# Uncomment for Google Colab
# !pip install pandas numpy matplotlib seaborn pandas-datareader fredapi statsmodels scipy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import pandas_datareader.data as web
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

## 1. Fetch Macroeconomic Data

In [ ]:
# Define date range
start_date = '2000-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')

# Fetch key economic indicators
indicators = {
    'GDP': 'Real GDP',
    'UNRATE': 'Unemployment Rate',
    'CPIAUCSL': 'Consumer Price Index',
    'FEDFUNDS': 'Federal Funds Rate',
    'DGS10': '10-Year Treasury Rate'
}

data = {}
for code, name in indicators.items():
    try:
        data[name] = web.DataReader(code, 'fred', start_date, end_date)
        print(f"✓ {name} fetched")
    except Exception as e:
        print(f"✗ Error fetching {name}: {e}")

# Create DataFrame
df = pd.concat(data.values(), axis=1, keys=data.keys())
df.columns = df.columns.droplevel(1)

print(f"\nData shape: {df.shape}")
print(f"Date range: {df.index[0]} to {df.index[-1]}")

## 2. Visualize Multiple Time Series

In [ ]:
# Create subplots for each indicator
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.flatten()

for i, col in enumerate(df.columns):
    axes[i].plot(df.index, df[col], linewidth=2)
    axes[i].set_title(col, fontsize=12, fontweight='bold')
    axes[i].set_xlabel('Date', fontsize=10)
    axes[i].set_ylabel('Value', fontsize=10)
    axes[i].grid(True, alpha=0.3)

# Remove extra subplot
fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()

## 3. Time Series Decomposition

Decompose unemployment rate into trend, seasonal, and residual components.

In [ ]:
# Decompose unemployment rate
unemployment = df['Unemployment Rate'].dropna()

# Perform seasonal decomposition
decomposition = seasonal_decompose(unemployment, model='additive', period=12)

# Plot decomposition
fig, axes = plt.subplots(4, 1, figsize=(14, 10))

decomposition.observed.plot(ax=axes[0], title='Observed')
axes[0].set_ylabel('Observed')

decomposition.trend.plot(ax=axes[1], title='Trend')
axes[1].set_ylabel('Trend')

decomposition.seasonal.plot(ax=axes[2], title='Seasonal')
axes[2].set_ylabel('Seasonal')

decomposition.resid.plot(ax=axes[3], title='Residual')
axes[3].set_ylabel('Residual')

plt.suptitle('Seasonal Decomposition of Unemployment Rate', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

## 4. Stationarity Test

Test if the unemployment rate series is stationary using the Augmented Dickey-Fuller test.

In [ ]:
def test_stationarity(series, name):
    """Perform Augmented Dickey-Fuller test"""
    result = adfuller(series.dropna())
    
    print(f'\nAugmented Dickey-Fuller Test for {name}:')
    print(f'ADF Statistic: {result[0]:.6f}')
    print(f'p-value: {result[1]:.6f}')
    print(f'Number of Lags Used: {result[2]}')
    print(f'Number of Observations: {result[3]}')
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'   {key}: {value:.3f}')
    
    if result[1] < 0.05:
        print(f'\nConclusion: The series is stationary at 5% significance level.')
    else:
        print(f'\nConclusion: The series is non-stationary at 5% significance level.')
    
    return result

# Test original series
test_stationarity(unemployment, 'Unemployment Rate')

# Test differenced series
unemployment_diff = unemployment.diff().dropna()
test_stationarity(unemployment_diff, 'Differenced Unemployment Rate')

## 5. Inflation Analysis

In [ ]:
# Calculate year-over-year inflation rate from CPI
cpi = df['Consumer Price Index'].dropna()
inflation = cpi.pct_change(periods=12) * 100

# Plot inflation rate
plt.figure(figsize=(14, 6))
plt.plot(inflation.index, inflation, linewidth=2, color='red')
plt.axhline(y=2, color='green', linestyle='--', alpha=0.7, label='2% Target')
plt.title('US Inflation Rate (Year-over-Year % Change in CPI)', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Inflation Rate (%)', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Average inflation rate: {inflation.mean():.2f}%")
print(f"Maximum inflation rate: {inflation.max():.2f}% on {inflation.idxmax().date()}")
print(f"Minimum inflation rate: {inflation.min():.2f}% on {inflation.idxmin().date()}")

## 6. Phillips Curve Analysis

Examine the relationship between inflation and unemployment (Phillips Curve).

In [ ]:
# Merge inflation and unemployment data
phillips_data = pd.DataFrame({
    'Inflation': inflation,
    'Unemployment': unemployment
}).dropna()

# Create scatter plot
plt.figure(figsize=(10, 8))
plt.scatter(phillips_data['Unemployment'], phillips_data['Inflation'], 
           alpha=0.5, s=50)

# Add trend line
z = np.polyfit(phillips_data['Unemployment'], phillips_data['Inflation'], 1)
p = np.poly1d(z)
plt.plot(phillips_data['Unemployment'], p(phillips_data['Unemployment']), 
        'r--', linewidth=2, label=f'Trend: y = {z[0]:.2f}x + {z[1]:.2f}')

plt.title('Phillips Curve: Inflation vs Unemployment', fontsize=16, fontweight='bold')
plt.xlabel('Unemployment Rate (%)', fontsize=12)
plt.ylabel('Inflation Rate (%)', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate correlation
correlation = phillips_data.corr().iloc[0, 1]
print(f"\nCorrelation between inflation and unemployment: {correlation:.3f}")

## 7. Interest Rate Analysis

In [ ]:
# Compare Fed Funds Rate and 10-Year Treasury
rates = df[['Federal Funds Rate', '10-Year Treasury Rate']].dropna()

# Plot both rates
plt.figure(figsize=(14, 6))
plt.plot(rates.index, rates['Federal Funds Rate'], label='Federal Funds Rate', linewidth=2)
plt.plot(rates.index, rates['10-Year Treasury Rate'], label='10-Year Treasury Rate', linewidth=2)
plt.title('US Interest Rates', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Rate (%)', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate yield curve (spread)
spread = rates['10-Year Treasury Rate'] - rates['Federal Funds Rate']

plt.figure(figsize=(14, 6))
plt.plot(spread.index, spread, linewidth=2, color='purple')
plt.axhline(y=0, color='red', linestyle='--', alpha=0.5)
plt.title('Yield Curve Spread (10Y Treasury - Fed Funds Rate)', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Spread (%)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nAverage spread: {spread.mean():.2f}%")
print(f"Current spread: {spread.iloc[-1]:.2f}%")

## Summary

This notebook demonstrated:
- Fetching multiple economic indicators from FRED
- Time series decomposition
- Stationarity testing
- Inflation analysis
- Phillips Curve visualization
- Interest rate analysis

Next steps:
- Try forecasting models (ARIMA, Prophet)
- Explore other economic relationships
- Compare with international data